In [1]:
import rospy
import rosbag
import pandas as pd
'''
需求分析：
通过rosbag保存了odom、imu的数据，现在想把这些数据提出去来进行分析，而不是在ros的框架下分析
1. 写一个话题订阅者，当rosbag发布话题时，订阅者触发回调函数，保存话题内容为CSV
    - 有至少两个话题：\imu, \odom，每个话题有一个订阅者
    - 我得知道消息类型，才能订阅，才知道怎么保存
2. 不必了，rosbag文件直接就能读数据，不用通过发布-订阅模型
    - 用哪个？odom or robot_pose_ekf/odom_combined? odom_combined
    - read_messages api怎么用？done

- 小车坐标系？
- 四元数和旋转角的转化？
'''

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [2]:
bag_file = "/home/dingrong/Code/robot_nav/dynamic_wall_para3_2023-04-08-16-58-29.bag"
bag = rosbag.Bag(bag_file, 'r')
bag_data = bag.read_messages()
imu_path = f"dynamic_wall_para3_imu.csv"
odom_path = f"dynamic_wall_para3_odom.csv"


In [3]:
odom_dict = {
    'stamp': [],
    'position_x': [],
    'position_y': [],
    'position_z': [],
    'orientation_x': [],
    'orientation_y': [],
    'orientation_z': [],
    'orientation_w': []
}
imu_dict = {
    'stamp': [],
    'angular_velocity_x': [],
    'angular_velocity_y': [],
    'angular_velocity_z': [],
    'linear_acceleration_x': [],
    'linear_acceleration_y': [],
    'linear_acceleration_z': []
}
for topic, msg, t in bag_data:
    if topic == '/imu':
        imu_dict['stamp'].append(msg.header.stamp.to_sec())
        imu_dict['angular_velocity_x'].append(msg.angular_velocity.x)
        imu_dict['angular_velocity_y'].append(msg.angular_velocity.y)
        imu_dict['angular_velocity_z'].append(msg.angular_velocity.z)
        imu_dict['linear_acceleration_x'].append(msg.linear_acceleration.x)
        imu_dict['linear_acceleration_y'].append(msg.linear_acceleration.y)
        imu_dict['linear_acceleration_z'].append(msg.linear_acceleration.z)
    if topic == '/robot_pose_ekf/odom_combined':
        odom_dict['stamp'].append(msg.header.stamp.to_sec())
        odom_dict['position_x'].append(msg.pose.pose.position.x)
        odom_dict['position_y'].append(msg.pose.pose.position.y)
        odom_dict['position_z'].append(msg.pose.pose.position.z)
        odom_dict['orientation_x'].append(msg.pose.pose.orientation.x)
        odom_dict['orientation_y'].append(msg.pose.pose.orientation.y)
        odom_dict['orientation_z'].append(msg.pose.pose.orientation.z)
        odom_dict['orientation_w'].append(msg.pose.pose.orientation.w)


In [4]:
imu_df = pd.DataFrame(imu_dict)
odom_df = pd.DataFrame(odom_dict)

In [5]:
imu_df.to_csv(imu_path, index=False)
odom_df.to_csv(odom_path, index=False)